# Installing Dependencies

Note this only works in a linux environment for some reason, I haven't tried with WSL but I assume it works with that. Needs a lot of RAM so I ran this on Google Colab with higher ram compute engines.

In [ ]:
! pip uninstall -y torch torchvision torchaudio
! pip cache purge

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Files removed: 0


In [ ]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [ ]:
! pip install --upgrade "transformers<4.30.0" peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 37.3 MB/s et

In [ ]:
! pip install highway-env
! pip install agilerl
! pip install pettingzoo

Set Up Simmulation Env and Agents

In [ ]:
import gymnasium as gym
import torch
import highway_env
from pettingzoo import ParallelEnv
from agilerl.vector.pz_async_vec_env import AsyncPettingZooVecEnv
from agilerl.utils.utils import create_population
from agilerl.components.multi_agent_replay_buffer import MultiAgentReplayBuffer
from agilerl.hpo.tournament import TournamentSelection
from agilerl.hpo.mutation import Mutations
from agilerl.training.train_multi_agent_off_policy import train_multi_agent_off_policy
import inspect
import numpy as np
from gymnasium.spaces import Box

[2025-09-23 17:28:59,579] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.


In [ ]:
print(inspect.signature(train_multi_agent_off_policy))

(env: pettingzoo.utils.env.ParallelEnv | agilerl.vector.pz_async_vec_env.AsyncPettingZooVecEnv, env_name: str, algo: str, pop: List[agilerl.algorithms.maddpg.MADDPG | agilerl.algorithms.matd3.MATD3], memory: agilerl.components.multi_agent_replay_buffer.MultiAgentReplayBuffer, sum_scores: bool = True, INIT_HP: Optional[Dict[str, Any]] = None, MUT_P: Optional[Dict[str, Any]] = None, swap_channels: bool = False, max_steps: int = 50000, evo_steps: int = 25, eval_steps: Optional[int] = None, eval_loop: int = 1, learning_delay: int = 0, target: Optional[float] = None, tournament: Optional[agilerl.hpo.tournament.TournamentSelection] = None, mutation: Optional[agilerl.hpo.mutation.Mutations] = None, checkpoint: Optional[int] = None, checkpoint_path: Optional[str] = None, overwrite_checkpoints: bool = False, save_elite: bool = False, elite_path: Optional[str] = None, wb: bool = False, verbose: bool = True, accelerator: Optional[accelerate.accelerator.Accelerator] = None, wandb_api_key: Optional

In [ ]:
config = {
    "observation": {
        "type": "MultiAgentObservation",
        "observation_config": {
            "type": "Kinematics",
            "features": ["presence", "x", "y", "vx", "vy", "cos_h", "sin_h", "heading"],
            "observe_intentions": False
        }
    },
    "controlled_vehicles": 2,
    "vehicles_count" : 10,
    "action": {
        "type": "MultiAgentAction",
        "action_config": {
            "type": "ContinuousAction",
        }
    },
    "offroad_terminal": True,
    "on_road_reward": 3.0,
    "duration": 25
}

In [ ]:
class IntersectionParallel(ParallelEnv):
    metadata = {"render_modes": ["rgb_array"]}
    def __init__(self, config, render_mode=None):
        self.render_mode = render_mode
        self.env = gym.make("intersection-v1", render_mode=render_mode)
        self.env.unwrapped.config.update(config)
        self.agents = [f"agent{i}" for i in range(config["controlled_vehicles"])]
        self.possible_agents = list(self.agents)
        obs, _ = self.env.reset()
        self._obs_shape = obs[0].flatten().shape
    def reset(self, **kwargs):
        tpl, _ = self.env.reset(**kwargs)
        return ({a: tpl[i].flatten() for i, a in enumerate(self.agents)},
            {a: {} for a in self.agents})
    def step(self, action_dict):
        actiontpl = tuple(action_dict[a] for a in self.agents)
        tpl, rews, term, trunc, _ = self.env.step(actiontpl)
        obs = {a: tpl[i].flatten() for i, a in enumerate(self.agents)}
        reward  = {a: rews for a in self.agents}
        terminated = {a: term for a in self.agents}
        truncated  = {a: trunc for a in self.agents}
        tpl, rews, term, trunc, _ = self.env.step(actiontpl)
        obs = {a: tpl[i].flatten() for i, a in enumerate(self.agents)}
        return obs, reward, terminated, truncated, {a:{} for a in self.agents}
    def observation_space(self, agent):
        return Box(low=-np.inf, high=np.inf, shape=self._obs_shape, dtype=np.float32)
    def action_space(self, agent):
        return self.env.action_space[0]

In [ ]:
num_envs = 1

In [ ]:
env = AsyncPettingZooVecEnv([lambda config = config: IntersectionParallel(config, render_mode="rgb_array") for _ in range(num_envs)])
obs, _ = env.reset()
print({a: obs[a].shape for a in env.agents})
print(env.observation_space("agent0"))

{'agent0': (1, 40), 'agent1': (1, 40)}
Box(-inf, inf, (1, 40), float32)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NET_CONFIG = {"head_config": {"hidden_size": [32, 32]}}
INIT_HP = {
    "CHANNELS_LAST": False,
    "BATCH_SIZE": 64,
    "LR_ACTOR": 3e-4,
    "LR_CRITIC":3e-4,
    "GAMMA":0.99,
    "MEMORY_SIZE":50_000,
    "LEARN_STEP":100,
    "TAU":0.005,
    "POLICY_FREQ":2,
    "POP_SIZE":1,
    "AGENT_IDS": env.agents,
}

In [ ]:
from agilerl.algorithms.core.registry import RLParameter, HyperparameterConfig

In [ ]:
hp_config = HyperparameterConfig(
    lr_actor=  RLParameter(min=1e-4, max=1e-3),
    lr_critic=  RLParameter(min=1e-4, max=1e-3),
    batch_size= RLParameter(min=16, max=256, dtype=int),
    learn_step= RLParameter(min=50, max=200, dtype=int),
)

In [ ]:
observation_spaces = [env.single_observation_space(a) for a in env.agents]
action_spaces      = [env.single_action_space(a)      for a in env.agents]
print(env.single_action_space(env.agents[0]))
pop = create_population(
    "MATD3",
    observation_spaces,
    action_spaces,
    NET_CONFIG,
    INIT_HP,
    hp_config,
    population_size=INIT_HP["POP_SIZE"],
    num_envs=num_envs,
    device=device,
)

Box(-1.0, 1.0, (2,), float32)


In [ ]:
memory = MultiAgentReplayBuffer(
    INIT_HP["MEMORY_SIZE"],
    field_names=["state","action","reward","next_state","done"],
    agent_ids=INIT_HP["AGENT_IDS"],
    device=device,
)

In [ ]:
tournament = TournamentSelection(
    tournament_size=1,
    elitism=True,
    population_size=INIT_HP["POP_SIZE"],
    eval_loop=1,
)
mutations = Mutations(
    no_mutation=0.5,
    architecture=0.1,
    parameters=0.1,
    rl_hp=0.5,
    mutation_sd=0.05,
    new_layer_prob=0.2,
    activation=0.1
)

Do Not Run Tournament Training Here. For enough steps, you must save and reload the agent.

In [ ]:
## IGNORE
trained_pop, fitnesses = train_multi_agent_off_policy(
    env=env,
    env_name="intersection-v1",
    algo="MATD3",
    pop=pop,
    memory=memory,
    INIT_HP=INIT_HP,
    swap_channels=INIT_HP["CHANNELS_LAST"],
    max_steps=50_000,
    evo_steps=10_000,
    eval_steps=None,
    eval_loop=1,
    learning_delay=1_000,
    target=None,
    tournament=tournament,
    mutation=mutations,
    wb=False,
    verbose=True,
)


Training...


 20%|##        | 10000/50000 [1:10:12<4:40:51,  2.37step/s]


DateTime, now, H:m:s-u 3 : 45 : 31 - 876509
Steps 2.371507876125224 per sec, 142.29047256751346 per min.

                --- Global Steps 10000 ---
                Fitness:	['4.00']
                Score:		[7.549819414297119]
                5 fitness avgs:	['4.00']
                10 score avgs:	['4.60']
                Agents:		[0]
                Steps:		[10000]
                Mutations:	['lr_actor']


 40%|####      | 20000/50000 [2:19:58<3:29:49,  2.38step/s]


DateTime, now, H:m:s-u 4 : 55 : 19 - 302870
Steps 4.776279916019961 per sec, 286.57679496119766 per min.

                --- Global Steps 20000 ---
                Fitness:	['5.00']
                Score:		[5.363377555597647]
                5 fitness avgs:	['4.50']
                10 score avgs:	['5.50']
                Agents:		[0]
                Steps:		[20000]
                Mutations:	['None']


...


DateTime, now, H:m:s-u 6 : 5 : 36 - 831141
Steps 7.113353561055902 per sec, 426.80121366335413 per min.

                --- Global Steps 30000 ---
                Fitness:	['12.00']
                Score:		[7.213111137042842]
                5 fitness avgs:	['7.00']
                10 score avgs:	['9.40']
                Agents:		[0]
                Steps:		[30000]
                Mutations:	['None']


 80%|########  | 40000/50000 [4:41:09<1:10:25,  2.37step/s]


DateTime, now, H:m:s-u 7 : 16 : 41 - 128185
Steps 9.380685110985295 per sec, 562.8411066591177 per min.

                --- Global Steps 40000 ---
                Fitness:	['4.00']
                Score:		[6.883307157715682]
                5 fitness avgs:	['6.25']
                10 score avgs:	['6.80']
                Agents:		[0]
                Steps:		[40000]
                Mutations:	['None']


100%|##########| 50000/50000 [5:51:57<  00:00,  2.37step/s]


DateTime, now, H:m:s-u 8 : 27 : 12 - 316914
Steps 11.817448278569659 per sec, 709.0468967141795 per min.

                --- Global Steps 50000 ---
                Fitness:	['5.00']
                Score:		[6.923719705766305]
                5 fitness avgs:	['6.00']
                10 score avgs:	['5.70']
                Agents:		[0]
                Steps:		[50000]
                Mutations:	['batch_size']


In [ ]:
# IGNORE
import os

save_dir = "./saved_agents"
os.makedirs(save_dir, exist_ok=True)

for idx, agent in enumerate(trained_pop):
    ckpt_path = os.path.join(save_dir, f"maddpg_agent_{idx}_50k_steps_reward_change2.pt")
    agent.save_checkpoint(ckpt_path)
    print(f"Saved agent {idx} → {ckpt_path}")


Saved agent 0 → ./saved_agents/maddpg_agent_0_50k_steps_reward_change2.pt


In [ ]:
# IGNORE

fitness_vals = [float(f) for f in fitnesses[-1]]
best_index = fitness_vals.index(max(fitness_vals))
best_agent = trained_pop[best_index]

print(f"Best Agent: ID={best_index}, Fitness={max(fitness_vals):.2f}")

Best Agent: ID=0, Fitness=5.00


# Training

In [ ]:
! pip install imageio

In [ ]:
from agilerl.algorithms.matd3 import MATD3
import time, os, json
from pathlib import Path
import sys
import matplotlib.pyplot as plt
from gymnasium import spaces
import numpy as np
import imageio.v2 as imageio

In [ ]:
RUN_NAME     = "intersection_matd3_run"
BASE_DIR     = Path('/content/drive/MyDrive/agilerl_runs') if 'google.colab' in sys.modules else Path('./agilerl_runs')
TEST_DIR     = BASE_DIR / "test_runs"
CKPT_DIR     = BASE_DIR / RUN_NAME

In [ ]:
def render_frame(frame):
    plt.imshow(frame)
    plt.axis("off")
    plt.show()

In [ ]:
for filename in os.listdir(CKPT_DIR):
    if filename.endswith("0.pt"):
        current_agent = filename.split(".")[0]
        print(current_agent)

agent_chunk_100000
agent_chunk_200000
agent_chunk_300000
agent_chunk_400000
agent_chunk_500000
agent_chunk_600000
agent_chunk_700000
agent_chunk_800000
agent_chunk_900000
agent_chunk_1000000
agent_chunk_1100000
agent_chunk_1200000
agent_chunk_1300000
agent_chunk_1400000
agent_chunk_1500000
agent_chunk_1600000
agent_chunk_1700000
agent_chunk_1800000
agent_chunk_1900000
agent_chunk_2000000


In [ ]:
scores = {}
TEST_DIR = Path(TEST_DIR)
TEST_DIR.mkdir(parents=True, exist_ok=True)
for filename in os.listdir(CKPT_DIR):
    if filename.endswith("0.pt"):
        current_agent = filename.split(".")[0]
        print(current_agent, "Test:")
        agent = MATD3.load(CKPT_DIR / filename, device="cpu")

        run_dir = TEST_DIR / current_agent
        run_dir.mkdir(parents=True, exist_ok=True)
        frames_dir = run_dir / "frames"
        frames_dir.mkdir(parents=True, exist_ok=True)

        jsonl_path = run_dir / "logs.jsonl"
        summary_path = run_dir / "summary.json"

        env = gym.make(
            "intersection-v1",
            config = config,
            render_mode="rgb_array"
        )
        obs, info = env.reset()
        done = False

        agent_ids = [f"agent{i}" for i in range(len(obs))]
        obs_dim = obs[0].flatten().shape[0]
        act_dim = env.action_space[0].shape[0]

        # Set correct spaces
        agent.observation_space = {
            aid: spaces.Box(low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float32)
            for aid in agent_ids
        }
        agent.action_space = {
            aid: spaces.Box(low=-1.0, high=1.0, shape=(act_dim,), dtype=np.float32)
            for aid in agent_ids
        }
        culminated = 0.0
        step_idx = 0
        with open(jsonl_path, "w") as f_log:
            while not done:
                obs_dict = {aid: obs[i].flatten() for i, aid in enumerate(agent_ids)}
                # print(obs_dict)

                actions_dict = agent.get_action(obs_dict)[0]
                # print(actions_dict)

                
                actions = tuple(tuple(actions_dict[aid][0]) for aid in agent_ids)

                # print("Agent actions:", actions)

                obs, reward, done, truncated, info = env.step(actions)
                # print("info:", info)
                # print("reward:", reward)

                if isinstance(reward, (list, tuple, np.ndarray)):
                    step_reward = float(np.mean(reward))
                    per_agent_reward = {aid: float(reward[i]) for i, aid in enumerate(agent_ids)}
                else:
                    step_reward = float(reward)
                    per_agent_reward = {aid: step_reward for aid in agent_ids}
                culminated += step_reward
                frame = env.render()
                imageio.imwrite(frames_dir / f"step_{step_idx:04d}.png", frame)
                # render_frame(frame)

                log_record = {
                    "step" : step_idx,
                    "observations": {aid: obs[i].flatten().astype(float).tolist()
                                     for i, aid in enumerate(agent_ids)},
                    "actions": {aid: list(map(float, actions[i])) for i, aid in enumerate(agent_ids)},
                    "reward" : culminated,
                    "rewards_per_agent": per_agent_reward
                }
                f_log.write(json.dumps(log_record) + "\n")

                step_idx += 1

                time.sleep(0.01)
        scores[current_agent] = culminated

        env.close()

        with open(summary_path, "w", encoding="utf-8") as f_sum:
            json.dump({
                "agent": current_agent,
                "checkpoint_file": str(filename),
                "steps": step_idx,
                "episode_return": culminated,
                "frames_dir": str(frames_dir),
                "logs_jsonl": str(jsonl_path),
            }, f_sum, indent=2)

        print(f"Saved run to: {run_dir}")


agent_chunk_100000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_100000
agent_chunk_200000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_200000
agent_chunk_300000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_300000
agent_chunk_400000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_400000
agent_chunk_500000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_500000
agent_chunk_600000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_600000
agent_chunk_700000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_700000
agent_chunk_800000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_800000
agent_chunk_900000 Test:
Saved run to: /content/drive/MyDrive/agilerl_runs/test_runs/agent_chunk_900000
agent_chunk_1000000 Test:
Saved run to: /content/drive/MyDrive/a

In [ ]:
scores

{'agent_chunk_100000': 1.6456986069679276,
 'agent_chunk_200000': 14.5,
 'agent_chunk_300000': 10.5,
 'agent_chunk_400000': 6.987231135368347,
 'agent_chunk_500000': 6.622776895761491,
 'agent_chunk_600000': 10.871216207742682,
 'agent_chunk_700000': 10.235796824097626,
 'agent_chunk_800000': 4.9676960110664385,
 'agent_chunk_900000': 5.0,
 'agent_chunk_1000000': 4.462341204285623,
 'agent_chunk_1100000': 3.5,
 'agent_chunk_1200000': 8.5,
 'agent_chunk_1300000': 7.0,
 'agent_chunk_1400000': 5.0,
 'agent_chunk_1500000': 3.5,
 'agent_chunk_1600000': 5.0,
 'agent_chunk_1700000': 1.5,
 'agent_chunk_1800000': 10.5,
 'agent_chunk_1900000': 10.5,
 'agent_chunk_2000000': 2.0}

# Checkpoint and Chunked Training


In [ ]:
import re

In [ ]:
from pickle import TRUE
# --- Colab/paths + high-level training plan ---
import os, sys, json, glob, time
from pathlib import Path

# Auto-mount Google Drive if running in Colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    if not Path('/content/drive').is_dir():
        drive.mount('/content/drive')

RUN_NAME     = "intersection_matd3_run"
BASE_DIR     = Path('/content/drive/MyDrive/agilerl_runs') if 'google.colab' in sys.modules else Path('./agilerl_runs')
CKPT_DIR     = BASE_DIR / RUN_NAME
CKPT_DIR.mkdir(parents=True, exist_ok=True)

TOTAL_STEPS  = 2_000_000    # <- overall target
CHUNK_STEPS  = 100_000      # <- per-call chunk size
CKPT_EVERY   = 50_000       # <- save mid-chunk checkpoints too
SAVE_MEMORY  = False         # <- turn off if saving the buffer causes issues

print(f"Checkpoints will be saved under: {CKPT_DIR}")

Mounted at /content/drive
Checkpoints will be saved under: /content/drive/MyDrive/agilerl_runs/intersection_matd3_run


In [ ]:
# --- Helper functions (agent + replay buffer + checkpoint discovery) ---
import torch

# Map your algo name to the correct class for load()
_ALGO_LOADERS = {}
try:
    from agilerl.algorithms.matd3 import MATD3
    _ALGO_LOADERS["MATD3"] = MATD3
except Exception:
    pass
try:
    from agilerl.algorithms.maddpg import MADDPG
    _ALGO_LOADERS["MADDPG"] = MADDPG
except Exception:
    pass

def latest_checkpoint_path(dirpath: Path):
    def extract_num(p: Path):
        match = re.search(r"agent_chunk_(\d+)\.pt", p.name)
        return int(match.group(1)) if match else -1

    # Prefer chunked checkpoints first
    candidates = sorted(dirpath.glob("agent_chunk_*.pt"), key=extract_num)
    if not candidates:
        # Fall back to any *.pt, sort by modification time
        candidates = sorted(dirpath.glob("*.pt"), key=lambda p: p.stat().st_mtime)

    return str(candidates[-1]) if candidates else None

def save_population_single(pop, step_total: int):
    # With POP_SIZE=1 in this notebook, pop[0] is the agent
    agent = pop[0]
    out = CKPT_DIR / f"agent_chunk_{step_total}.pt"
    agent.save_checkpoint(str(out))
    print(f"Saved agent checkpoint -> {out}")

def save_memory_safe(memory_obj):
    # First try torch.save (fastest if picklable)
    try:
        torch.save(memory_obj, CKPT_DIR / "replay_buffer.pt")
        print("Saved replay buffer -> replay_buffer.pt")
        return
    except Exception as e:
        print("torch.save(memory) failed:", e)
    # Fallback: try Python pickle (may still fail depending on internals)
    import pickle
    try:
        with open(CKPT_DIR / "replay_buffer.pkl", "wb") as f:
            pickle.dump(memory_obj, f)
        print("Saved replay buffer (pickle) -> replay_buffer.pkl")
    except Exception as e:
        print("Pickle save of replay buffer also failed:", e)

def load_memory_safe():
    # Try torch first
    pt = CKPT_DIR / "replay_buffer.pt"
    if pt.exists():
        try:
            return torch.load(pt, map_location='cpu')
        except Exception as e:
            print("torch.load(replay_buffer.pt) failed:", e)
    # Try pickle
    import pickle
    pkl = CKPT_DIR / "replay_buffer.pkl"
    if pkl.exists():
        try:
            with open(pkl, "rb") as f:
                return pickle.load(f)
        except Exception as e:
            print("pickle.load(replay_buffer.pkl) failed:", e)
    return None

def load_latest_agent(algo_name: str, device='cpu'):
    ckp = latest_checkpoint_path(CKPT_DIR)
    if not ckp:
        return None, None
    cls = _ALGO_LOADERS.get(algo_name)
    if not cls:
        print(f"No loader registered for algo '{algo_name}'. Update _ALGO_LOADERS mapping.")
        return None, None
    agent = cls.load(ckp, device=device)
    print(f"Loaded agent from {ckp}")
    return agent, ckp

def load_steps_done():
    meta = CKPT_DIR / "meta.json"
    if meta.exists():
        try:
            return json.loads(meta.read_text()).get("steps_done", 0)
        except Exception:
            return 0
    return 0

def save_steps_done(steps):
    (CKPT_DIR / "meta.json").write_text(json.dumps({"steps_done": int(steps)}, indent=2))
    print(f"Updated meta.json -> steps_done={steps}")

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- run this after you've created env, pop, memory, etc. ---
# Requires that variables `INIT_HP`, `env`, `pop`, `memory`, `tournament`, `mutations` exist 
assert all(k in globals() for k in ('INIT_HP','env','pop','memory')), \
    "Run the earlier setup cells (env/pop/memory) before resuming."

algo_name = "MATD3"  # this notebook uses MATD3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Try to load the latest agent checkpoint (optional; if none found, keeps current pop)
loaded_agent, ckppath = load_latest_agent(algo_name, device=device)
if loaded_agent is not None:
    pop = [loaded_agent]  

# Try to load replay buffer
if SAVE_MEMORY:
    loaded_mem = load_memory_safe()
    if loaded_mem is not None:
        memory = loaded_mem
        print("Replay buffer restored.")

Loaded agent from /content/drive/MyDrive/agilerl_runs/intersection_matd3_run/agent_chunk_1900000.pt


In [ ]:
# --- Chunked training orchestrator ---
from agilerl.training.train_multi_agent_off_policy import train_multi_agent_off_policy

steps_done = load_steps_done()
print(f"Starting (or resuming) at steps_done={steps_done} / {TOTAL_STEPS}")

while steps_done < TOTAL_STEPS:
    steps_this = min(CHUNK_STEPS, TOTAL_STEPS - steps_done)
    target_cuml = steps_done + steps_this
    print(f"\n>>> Training chunk: {steps_this} steps (target cumulative: {steps_done + steps_this})")
    trained_pop, fitnesses = train_multi_agent_off_policy(
        env=env,
        env_name="intersection-v1",
        algo="MATD3",
        pop=pop,
        memory=memory,
        INIT_HP=INIT_HP,
        swap_channels=INIT_HP["CHANNELS_LAST"],
        max_steps=target_cuml,
        evo_steps=50_000,
        eval_steps=None,
        eval_loop=1,
        learning_delay=1_000,
        target=None,
        tournament=tournament,
        mutation=mutations,
        checkpoint=CKPT_EVERY,
        checkpoint_path=str(CKPT_DIR),
        overwrite_checkpoints=False,
        save_elite=True,
        elite_path=str(CKPT_DIR / "elite.pt"),
        wb=False,
        verbose=True,
    )

    # Update the working population 
    pop = trained_pop

    # Save agent 
    save_population_single(pop, steps_done + steps_this)
    if SAVE_MEMORY:
        save_memory_safe(memory)

    steps_done = target_cuml
    save_steps_done(steps_done)
    save_population_single(pop, steps_done)

print("\nTraining complete")

Starting (or resuming) at steps_done=1900000 / 2000000

>>> Training chunk: 100000 steps (target cumulative: 2000000)

Training...




🚀 Training Progress │   0% │                      │ 0/2000000 steps │ ⏱️ 00:00 │ ⏳ ? │ ?step/s

🚀 Training Progress │   2% │ ▌                    │ 50000/2000000 steps │ ⏱️ 10:07:04 │ ⏳ 394:35:47 │  1.37step/s
/usr/local/lib/python3.12/dist-packages/agilerl/hpo/mutation.py:451: UserWarning: Activation mutations are not supported for MATD3.
  warnings.warn(





🚀 Training Progress │   2% │ ▌                    │ 50000/2000000 steps │ ⏱️ 3:57:35 │ ⏳ 153:24:04 │  3.53step/s
🚀 Training Progress │   0% │                      │ 0/2000000 steps │ ⏱️ 3:57:58 │ ⏳ ? │ ?step/s

         Global Steps 50000        
Fitness:	['8.00']
Score:		[7.44753045200153]
5 fitness avgs:	['8.20']
10 score avgs:	['6.80']
Agents:		[0]
Steps:		[2000000]
Mutations:	['None']


🚀 Training Progress │   2% │ ▌                    │ 50000/2000000 steps │ ⏱️ 3:58:13 │ ⏳ 154:50:27 │  3.50step/s


Saved agent checkpoint -> /content/drive/MyDrive/agilerl_runs/intersection_matd3_run/agent_chunk_2000000.pt
Updated meta.json -> steps_done=2000000
Saved agent checkpoint -> /content/drive/MyDrive/agilerl_runs/intersection_matd3_run/agent_chunk_2000000.pt

✅ Training complete!
